# TabPy Deployment Script
This notebook connects to a running TabPy server and deploys our `ReadmissionRiskPredictor` function. The function uses a pre-trained scikit-learn model to make predictions.

In [ ]:
# This script sets up a TabPy server and registers a simple function
# for making predictions with a pre-trained scikit-learn model.

from tabpy.tabpy_tools.client import Client
import joblib
import pandas as pd
import numpy as np

# --- 1. Load the Trained Model ---
# Load the saved logistic regression model
try:
    model = joblib.load('patient_readmission_prediction_model.pkl')
    print("Trained model loaded successfully.")
except FileNotFoundError:
    print("Error: 'patient_readmission_prediction_model.pkl' not found.")
    print("Please ensure the trained model file is in the same directory as this script.")
    exit()

# --- 2. Set up the TabPy Client and Connect to Server ---
# Assuming TabPy is running on your machine on the default port (9004)
# You can start the server by running 'tabpy' in your command prompt.

try:
    # Correct the endpoint format to include http:// and the port, including the trailing slash
    client = Client('http://localhost:9004/')
    print("Successfully connected to the TabPy server.")
except ConnectionRefusedError:
    print("Could not connect to the TabPy server.")
    print("Please ensure you have started the TabPy server by running 'tabpy' in your terminal.")
    exit()

# --- 3. Define the Prediction Function ---
# This is the function that Tableau will call. It will receive data as a list from Tableau.

# Define the list of features your model expects (should match x_cols from training)
# This list is needed to reconstruct the DataFrame from the list input.
model_features = [
    # Strong indicators
    'age_[50-60)', 'age_[60-70)', 'age_[70-80)', 'age_[80-90)', 'age_[90-100)'
    , 'diabetes_med_yes'
    , 'has_outpatient_visit', 'has_inpatient_visit', 'has_emergency_visit'
    , 'diag_1_Diabetes', 'diag_1_Digestive', 'diag_1_Injury', 'diag_1_Musculoskeletal', 'diag_1_Other', 'diag_1_Respiratory'
    , 'diag_2_Diabetes', 'diag_2_Digestive', 'diag_2_Injury', 'diag_2_Musculoskeletal', 'diag_2_Other', 'diag_2_Respiratory'
    , 'diag_3_Diabetes', 'diag_3_Digestive', 'diag_3_Injury', 'diag_3_Musculoskeletal', 'diag_3_Other', 'diag_3_Respiratory'
    , 'change_yes'
    , 'A1Ctest_no', 'A1Ctest_normal'
    , 'glucose_test_no', 'glucose_test_normal'
]


def predict_readmission_risk(input_data_list):
    """
    Predicts the readmission risk based on a set of patient features.
    The function takes a list of data from Tableau, converts it to a DataFrame,
    and returns a list of predicted readmission probabilities.

    Args:
        input_data_list (list): List containing patient features sent from Tableau.

    Returns:
        list: A list of predicted readmission probabilities.
    """

    # --- Convert list input to DataFrame ---
    # Tableau sends data as a list of lists or a single list if only one row.
    # Convert the list input into a pandas DataFrame with the correct column names.
    try:
        # Ensure input_data_list is a list of lists if it's a single list
        if not isinstance(input_data_list[0], list):
             input_data_list = [input_data_list]

        # Check if the number of columns in the input list matches the expected number of features
        if len(input_data_list[0]) != len(model_features):
            raise ValueError(f"Mismatch in number of columns. Expected {len(model_features)}, but received {len(input_data_list[0])}")

        input_df = pd.DataFrame(input_data_list, columns=model_features)
        print(f'Converted input list to DataFrame:\n{input_df.head()}')

    except Exception as e:
        print(f"Error converting input list to DataFrame: {e}")
        return [None] * len(input_data_list)


    # --- Preprocessing Steps (Apply the same transformations as in training) ---
    # Now that we have a DataFrame, we can apply reindexing.
    try:
        # Ensure the input data has the same columns in the same order as the training data
        # This reindexing should now work since input_df is a DataFrame
        processed_data = input_df.reindex(columns=model_features, fill_value=0)
        print(f'Processed data after reindexing:\n{processed_data.head()}')

    except Exception as e:
        # Handle potential errors during reindexing
        print(f"Error during data reindexing: {e}")
        # Depending on the error, you might return an error message or default values
        return [None] * len(input_data_list)


    # --- Make Predictions ---
    # Use the loaded pre-trained model to make predictions
    # We use predict_proba to get the probability of the positive class (readmitted)
    # [:, 1] selects the probabilities for the positive class (readmitted=1)
    try:
        predictions = model.predict_proba(processed_data)[:, 1]
        print(f'Raw predictions from model: {predictions[:5]}') # Print first 5 predictions

    except Exception as e:
        print(f"Error during model prediction: {e}")
        return [None] * len(input_data_list)


    # Return predictions as a list
    return predictions.tolist()

# --- 4. Deploy the Function to TabPy ---
# This makes the function available to Tableau.
# The `name` is how you will refer to it in Tableau.

client.deploy(
    'ReadmissionRiskPredictor',
    predict_readmission_risk, # Pass the function as the first argument
    'Predicts patient readmission risk using a trained scikit-learn Logistic Regression model.',
    override=True
)

print("\n'ReadmissionRiskPredictor' function successfully deployed to TabPy.")
print("You can now connect to your TabPy server from Tableau and use this function.")

Trained model loaded successfully.
Successfully connected to the TabPy server.

'ReadmissionRiskPredictor' function successfully deployed to TabPy.
You can now connect to your TabPy server from Tableau and use this function.
